In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('/content/drive/MyDrive/AI/DL/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df = df.dropna()

In [4]:
X = df.drop('label',axis=1)

In [5]:
y = df['label']

In [6]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [7]:
X.shape, y.shape

((18285, 4), (18285,))

In [8]:
import tensorflow as tf


In [9]:
tf.__version__

'2.4.1'

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


In [11]:
voc_size = 5000

In [12]:
messages = X.copy()

In [13]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.stem.porter import PorterStemmer

In [18]:
ps = PorterStemmer()

In [19]:
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review =[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
one_hot_repr = [one_hot(word,voc_size) for word in corpus]

In [22]:
one_hot_repr

[[2410, 3627, 665, 4756, 1060, 918, 2861, 4010, 4451, 649],
 [2301, 2928, 3000, 4545, 2432, 434, 1318],
 [4751, 4421, 2546, 380],
 [4139, 4396, 2782, 4669, 880, 4383],
 [609, 2432, 161, 3572, 109, 3428, 2432, 2087, 128, 443],
 [3753,
  1021,
  3355,
  308,
  4608,
  4890,
  985,
  2921,
  3042,
  4747,
  1898,
  3360,
  3983,
  2894,
  1318],
 [4632, 3698, 1721, 156, 1023, 1066, 2046, 309, 209, 1292, 1239],
 [4259, 3977, 3643, 83, 1115, 1375, 4890, 4945, 209, 1292, 1239],
 [568, 4885, 4657, 1105, 2507, 3333, 549, 3189, 4890, 57],
 [2408, 2353, 2659, 107, 4491, 4269, 1317, 3040],
 [4193, 1543, 2430, 3202, 4767, 2922, 1300, 3050, 3208, 620, 2982],
 [4669, 1184, 1060, 3333, 4890, 1115],
 [1735, 4231, 2302, 1120, 1606, 3501, 296, 2121, 2218],
 [1280, 401, 4796, 1508, 3635, 1838, 1087, 209, 1292, 1239],
 [1436, 4117, 871, 1814, 4659, 209, 1292, 1239],
 [3013, 3231, 4756, 71, 3443, 3704, 2324, 1763, 4721, 493],
 [2235, 2629, 2928],
 [4241, 1404, 156, 2095, 4890, 82, 1900, 1318],
 [2844, 991,

In [23]:
sent_len=20

In [24]:
embeded_doc = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_len)


In [25]:
embeded_doc

array([[   0,    0,    0, ..., 4010, 4451,  649],
       [   0,    0,    0, ..., 2432,  434, 1318],
       [   0,    0,    0, ..., 4421, 2546,  380],
       ...,
       [   0,    0,    0, ...,  209, 1292, 1239],
       [   0,    0,    0, ..., 1939,  847, 1452],
       [   0,    0,    0, ..., 1046, 2332, 3730]], dtype=int32)

In [26]:
embeded_vector_feature = 40

In [27]:
model = Sequential()

In [28]:
model.add(Embedding(voc_size,embeded_vector_feature,input_length=sent_len))

In [29]:
model.add(Bidirectional(LSTM(100)))

In [30]:
model.add(Dropout(0.3))

In [31]:
model.add(Dense(1,activation='sigmoid'))

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [34]:
import numpy as np


In [35]:
final_X = np.array(embeded_doc)
final_y = np.array(y)


In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(final_X, final_y, test_size=0.33, random_state=43)

In [38]:
model.fit (X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 36s 16ms/step - loss: 0.4341 - accuracy: 0.7926 - val_loss: 0.1953 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1367 - accuracy: 0.9496 - val_loss: 0.1933 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0953 - accuracy: 0.9668 - val_loss: 0.2145 - val_accuracy: 0.9186
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0564 - accuracy: 0.9816 - val_loss: 0.2403 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0363 - accuracy: 0.9885 - val_loss: 0.3316 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0248 - accuracy: 0.9923 - val_loss: 0.3768 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0167 - accuracy: 0.9943 - val_loss: 0.4094 - val_accuracy: 0.918

In [39]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [41]:
confusion_matrix(y_test, y_pred)

array([[3158,  306],
       [ 200, 2371]])

In [42]:
accuracy_score(y_test, y_pred)

0.9161557580778791

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3464
           1       0.89      0.92      0.90      2571

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.92      0.92      6035

